In [49]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [50]:
# Creates the antecedents, or inputs, to decide the most efficient solution.
risk = ctrl.Antecedent(np.arange(0, 101, 1), 'risk')            # Risk of the generated action
time = ctrl.Antecedent(np.arange(0, 101, 1), 'time')            # Time to execute the action
energy = ctrl.Antecedent(np.arange(0, 101, 1), 'energy')        # Considers the current energy of the system.
complexity = ctrl.Antecedent(np.arange(0, 101, 1), 'complexity')# 
impact = ctrl.Antecedent(np.arange(0, 101, 1), 'impact')        # How much it would affect the mission

# Output variable to decide the best course of action, the fuzzy set of each action depends on the State (S) of the system
action = ctrl.Consequent(np.arange(0, 11, 1), 'action')

In [51]:
# Fuzzy sets for input variables, the sets are splitted in 3 different categories which are (low, medium, high)
risk.automf(3, variable_type='quantity')
time.automf(3, 'quantity')
energy.automf(3, 'quantity')
complexity.automf(3, 'quantity')
impact.automf(3, 'quantity')

In [52]:
# Create the possible actions for each state S
def actions(state):
    action.terms = {} # Clear the terms
    if state == "S1": # Normal
        action['remain_normal'] = fuzz.trapmf(action.universe, [0, 0, 8, 10])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [8, 9, 10, 10])
        return 
    elif state == "S2": # GPS Spoofing
        action['restore_attack'] = fuzz.trimf(action.universe, [0, 0, 3])
        action['change_route'] = fuzz.trimf(action.universe, [1, 3, 5])
        action['activate_redundancy'] = fuzz.trimf(action.universe, [3, 5, 7])
        action['use_occupancy_maps'] = fuzz.trimf(action.universe, [5, 7, 9])
        action['emergency_landing'] = fuzz.trimf(action.universe, [7, 10, 10])
        return
    elif state == "S3": # Jamming GPS
        action['change_altitude'] = fuzz.trapmf(action.universe, [0, 0, 2, 3])
        action['restore_system'] = fuzz.trapmf(action.universe, [1, 4, 5, 7])
        action['activate_redundancy'] = fuzz.trapmf(action.universe, [3, 6, 7, 9])
        action['emergency_landing'] = fuzz.trimf(action.universe, [7, 10, 10])
        return
    elif state == "S4": # Engine
        action['activate_redundancy'] = fuzz.trapmf(action.universe, [0, 0, 8, 10])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [8 ,9, 10, 10])
    elif state == "S5": # Aileron or elevator
        action['change_altitude'] = fuzz.trapmf(action.universe, [0, 0, 8, 10])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [8 ,9, 10, 10])
        return
    elif state == "S6": # Rudder
        action['navigation_adjustment'] = fuzz.trapmf(action.universe, [0, 0, 8, 10])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [8 ,9, 10, 10])
        return

In [53]:
# Create the rules for the fuzzy system
def rules(state):
    if state == "S1": # Normal
        return [
            ctrl.Rule(~risk['high'] | (~energy['low'] & ~time['low']), action['remain_normal']),
            ctrl.Rule(risk['high'] & (energy['low'] & time['low']) | ~impact['low'] & ~complexity['low'], action['emergency_landing']),
            ctrl.Rule(risk['average'] & energy['average'] & time['average'], action['remain_normal']),
            ctrl.Rule(risk['low'] & energy['high'] & time['high'], action['remain_normal']),
            ctrl.Rule(complexity['high'] & impact['average'], action['emergency_landing']),
            ctrl.Rule(energy['low'] & ~risk['high'], action['remain_normal']),
            ctrl.Rule(time['low'] & ~risk['high'], action['remain_normal']),
            ctrl.Rule(impact['high'] & ~risk['high'], action['emergency_landing'])
        ]
    elif state == "S2": # GPS Spoofing
        return [
            ctrl.Rule(risk['low'] & impact['low'] & complexity['low'], action['restore_attack']),
            ctrl.Rule(time['high'] & energy['average'] & impact['average'], action['change_route']),
            ctrl.Rule(time['average'] & energy['average'] & impact['average'], action['activate_redundancy']),
            ctrl.Rule(time['low'] & energy['average'] & impact['high'], action['use_occupancy_maps']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing']),
            ctrl.Rule(risk['average'] & complexity['average'], action['restore_attack']),
            ctrl.Rule(energy['high'] & impact['low'], action['change_route']),
            ctrl.Rule(complexity['high'] & impact['high'], action['use_occupancy_maps']),
            ctrl.Rule(risk['low'] & time['high'], action['activate_redundancy'])
        ]
    elif state == "S3": # Jamming GPS
        return [
            ctrl.Rule(risk['low'] & impact['low'] & complexity['low'], action['change_altitude']),
            ctrl.Rule(time['high'] & energy['average'] & impact['average'], action['restore_system']),
            ctrl.Rule(complexity['average'] & energy['average'] & time['average'], action['activate_redundancy']),
            ctrl.Rule((risk['high'] | complexity['high'] | impact['high']) & energy['low'] & time['low'], action['emergency_landing']),
            ctrl.Rule(risk['average'] & time['average'], action['change_altitude']),
            ctrl.Rule(energy['high'] & complexity['low'], action['restore_system']),
            ctrl.Rule(impact['high'] & time['low'], action['emergency_landing']),
            ctrl.Rule(risk['low'] & complexity['high'], action['activate_redundancy']),
        ]
    elif state == "S4": # Engine
        return [
            ctrl.Rule(complexity['average'] & energy['average'] & time['average'], action['activate_redundancy']),
            ctrl.Rule((risk['average'] | complexity['average']) & energy['high'], action['activate_redundancy']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing']),
            ctrl.Rule(risk['low'] & energy['average'], action['activate_redundancy']),
            ctrl.Rule(complexity['low'] & time['high'], action['activate_redundancy']),
            ctrl.Rule(impact['high'] & energy['average'], action['emergency_landing']),
            ctrl.Rule(time['low'] & risk['average'], action['activate_redundancy']),
            ctrl.Rule(energy['low'] & impact['high'], action['emergency_landing']),
            ctrl.Rule(complexity['high'] & time['average'], action['activate_redundancy'])
        ]
    elif state == "S5": # Aileron
        return [
            ctrl.Rule((risk['average'] | complexity['average']) & energy['high'], action['change_altitude']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing']),
            ctrl.Rule(risk['low'] & time['high'], action['change_altitude']),
            ctrl.Rule(impact['average'] & energy['average'], action['change_altitude']),
            ctrl.Rule(complexity['low'] & time['low'], action['emergency_landing']),
            ctrl.Rule(energy['average'] & risk['average'], action['change_altitude']),
            ctrl.Rule(time['average'] & impact['high'], action['emergency_landing'])
        ]
    elif state == "S6": # Rudder
        return [
            ctrl.Rule((risk['average'] | complexity['average']) & energy['high'], action['navigation_adjustment']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing']),
            ctrl.Rule(risk['low'] & time['high'], action['navigation_adjustment']),
            ctrl.Rule(impact['average'] & energy['average'], action['navigation_adjustment']),
            ctrl.Rule(complexity['low'] & time['low'], action['emergency_landing']),
            ctrl.Rule(energy['average'] & risk['average'], action['navigation_adjustment']),
            ctrl.Rule(time['average'] & impact['high'], action['emergency_landing'])
        ]

In [54]:
def get_action(state, risk_value, time_value, energy_value, complexity_value, impact_value):
    actions(state)
    rule_set = rules(state)
    action_ctrl = ctrl.ControlSystem(rule_set)
    action_simulation = ctrl.ControlSystemSimulation(action_ctrl)

    action_simulation.input['risk'] = risk_value
    action_simulation.input['time'] = time_value
    action_simulation.input['energy'] = energy_value
    action_simulation.input['complexity'] = complexity_value
    action_simulation.input['impact'] = impact_value

    action_simulation.compute()
    
    # Get the defuzzified output value
    action_value = action_simulation.output['action']

    #action.view(sim=action_simulation)
    #action_simulation.print_state()
    
    # Determine the action with the highest membership at this value
    action_memberships = {}
    for action_name in action.terms:
        action_memberships[action_name] = fuzz.interp_membership(action.universe, action[action_name].mf, action_value)
    
    # print(action_memberships)
    
    # Choses the action based on the highest membership value 
    chosen_action = max(action_memberships, key=action_memberships.get)
    return chosen_action

print(get_action(state= "S3", risk_value= 99, time_value= 1, energy_value= 40,complexity_value= 50, impact_value= 30))

activate_redundancy


In [55]:
import random
for i in range(0, 20):
    print(get_action(state="S2", 
                     risk_value=random.randint(0, 100), 
                     time_value=random.randint(0, 100), 
                     energy_value=random.randint(0, 100), 
                     complexity_value=random.randint(0, 100), 
                     impact_value=random.randint(0, 100)))


activate_redundancy
change_route
change_route
change_route
activate_redundancy
use_occupancy_maps
use_occupancy_maps
change_route
change_route
activate_redundancy
activate_redundancy
change_route
activate_redundancy
activate_redundancy
use_occupancy_maps
change_route
change_route
activate_redundancy
change_route
change_route


In [56]:
def change_arguments(state, risk_value, time_value, energy_value, complexity_value, impact_value, action):
    if state == "S2":
        if action == 'restore_attack':
            risk_value[0] += 5
            complexity_value[0] += 5
            impact_value[0] += 5
        elif action == 'change_route':
            time_value[0] -= 5
            energy_value[0] -= 5
            impact_value[0] += 15
        elif action == 'activate_redundancy':
            energy_value[0] -= 5
            complexity_value[0] += 10
            impact_value[0] += 5
        elif action == 'use_occupancy_maps':
            time_value[0] -= 10
            energy_value[0] -= 10
            impact_value[0] += 20
            risk_value[0] += 15

def abort(action):
    if action == 'emergency_landing':
        print("Emergency landing!")
        print("Aborting mission")
        return True
    return False

In [57]:
time_remaining = [900] # 15 minutes
energy_level = [100] # 100% of energy
risk_level = [0]
complexity_level =[0]
impact_level = [0]

actions_taken = []
while (time_remaining[0] > 0 and energy_level[0] > 0):
    state = "S1"
    if (time_remaining[0] == 600): # In minute 10, a GPS Spoofing happens
        state = "S2"
    elif (time_remaining[0] == 300):   # In minute 5, an Engine failure happens
        state = "S4"
    
    time_level = time_remaining[0]/9 # Percentage of time that remains

    # Stores the action taken in a tuple with the current state
    action_taken = get_action(state, risk_level[0], time_level, energy_level[0], complexity_level[0], impact_level[0])
    actions_taken.append((state, action_taken)) 
    
    change_arguments(state, risk_level, time_remaining, energy_level, complexity_level, impact_level, action_taken)

    print(time_remaining[0], energy_level[0], actions_taken[len(actions_taken) - 1])

    if abort(action_taken):   # If a emergency landing is needed, the mission is aborted
        break

    time_remaining[0] -= 1 # Reduces the time by 1 second
    energy_level[0] -= 0.027   # Reduces the energy by 0.027% per second (1 hour of battery life)
    risk_level[0] += 0.01 

900 100 ('S1', 'remain_normal')
899 99.973 ('S1', 'remain_normal')
898 99.946 ('S1', 'remain_normal')
897 99.919 ('S1', 'remain_normal')
896 99.892 ('S1', 'remain_normal')
895 99.865 ('S1', 'remain_normal')
894 99.838 ('S1', 'remain_normal')
893 99.81099999999999 ('S1', 'remain_normal')
892 99.78399999999999 ('S1', 'remain_normal')
891 99.75699999999999 ('S1', 'remain_normal')
890 99.72999999999999 ('S1', 'remain_normal')
889 99.70299999999999 ('S1', 'remain_normal')
888 99.67599999999999 ('S1', 'remain_normal')
887 99.64899999999999 ('S1', 'remain_normal')
886 99.62199999999999 ('S1', 'remain_normal')
885 99.59499999999998 ('S1', 'remain_normal')
884 99.56799999999998 ('S1', 'remain_normal')
883 99.54099999999998 ('S1', 'remain_normal')
882 99.51399999999998 ('S1', 'remain_normal')
881 99.48699999999998 ('S1', 'remain_normal')
880 99.45999999999998 ('S1', 'remain_normal')
879 99.43299999999998 ('S1', 'remain_normal')
878 99.40599999999998 ('S1', 'remain_normal')
877 99.37899999999998 

In [58]:
print(actions_taken)

[('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'), ('S1', 'remain_normal'),